In [1]:
!pip install  langchain langchain-community langchain-openai langchain-experimental wikipedia tiktoken py2neo streamlit neo4j langchain-core pyvis cairocffi igraph

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pandas-2.2.2-cp311-cp311-macosx_10_9_x86_64.whl.metadata (19 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.4 MB/s eta 0:00:00
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.6/983.6 kB 15.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 24.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.7/202.7 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 19.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 35.2 MB/s eta 0:00:0000:0

In [2]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import ConfigurableField, RunnableParallel, RunnablePassthrough

from langchain_openai import OpenAIEmbeddings
import igraph as ig
import os

try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [3]:
import pandas as pd
from py2neo import Graph
from pyvis.network import Network
import networkx as nx
import plotly.graph_objects as go

In [4]:
# --- Step 1: Connect to Neo4j (Replace with your credentials) ---
graph = Graph(auth=("neo4j", "12345678"), uri="bolt://localhost:7687")

In [5]:
# Clear existing data (Optional)
graph.run("MATCH (n) DETACH DELETE n")

(No data)

In [6]:
# --- Step 2: Load CSV Data ---
data1 = pd.read_csv("Financials.csv")
data1.columns = data1.columns.str.strip()
data1

,Segment,Country,Product,Discount Band,Units Sold,Manufacturing Price,sales price,Gross Sales,Discounts,Sales,COGS,Profit,Date,Month Number,Month Name,Year
0,Government,Canada,Carretera,None,"$1,618.50",$3.00,$20.00,"$30,000",NaN,"$32,370.00","$16,185.00","$16,185.00",1/1/2014,1,January,2014
1,Government,Germany,Carretera,None,"$1,321.00",$3.00,$20.00,"$26,420.00",NaN,"$26,420.00","$13,210.00","$13,210.00",1/1/2014,1,January,2014
2,Midmarket,France,Carretera,None,"$2,178.00",$3.00,$15.00,"$32,670.00",NaN,"$32,670.00","$21,780.00","$10,890.00",1/6/2014,6,June,2014
3,Midmarket,Germany,Carretera,None,$888.00,$3.00,$15.00,"$13,320.00",NaN,"$13,320.00","$8,880.00","$4,440.00",1/6/2014,6,June,2014
4,Midmarket,Mexico,Carretera,None,"$2,470.00",$3.00,$15.00,"$37,050.00",NaN,"$37,050.00","$24,700.00","$12,350.00",1/6/2014,6,June,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,Small Business,France,Amarilla,High,"$2,475.00",$260.00,$300.00,"$7,42,500.00","$1,11,375.00","$6,31,125.00","$6,18,750.00","$12,375.00",1/3/2014,3,March,2014
696,Small Business,Mexico,Amarilla,High,$546.00,$260.00,$300.00,"$1,63,800.00","$24,570.00","$1,39,230.00","$1,36,500.00","$2,730.00",1/10/2014,10,October,2014
697,Government,Mexico,Montana,High,"$1,368.00",$5.00,$7.00,"$9,576.00","$1,436.40","$8,139.60","$6,840.00","$1,299.60",1/2/2014,2,February,2014
698,Government,Canada,Paseo,High,$723.00,$10.00,$7.00,"$5,061.00",$759.15,"$4,301.85","$3,615.00",$686.85,1/4/2014,4,April,2014


In [7]:
len(data1)

700

In [8]:
# Create nodes (MERGE prevents duplicates)
for index, row in data1.iterrows():
    segment_name = row["Segment"]
    country_name = row["Country"]
    product_name = row["Product"]

    graph.run("MERGE (s:Segment {name: $segment_name})", segment_name=segment_name)
    graph.run("MERGE (c:Country {name: $country_name})", country_name=country_name)
    graph.run("MERGE (p:Product {name: $product_name})", product_name=product_name)

In [9]:
# Add Relationships and Properties
for index, row in data1.iterrows():
    product_name = row["Product"]
    country_name = row["Country"]
    sales_value = row['Sales']

    graph.run("MATCH (p:Product {name: $product_name}), (c:Country {name: $country_name}) CREATE (p)-[r:SOLD_IN {sales: $sales_value}]->(c)",
              product_name=product_name, country_name=country_name, sales_value=sales_value)

In [10]:
graph.run("MATCH (n)-[r]->(m) RETURN n, r, m")


n,r,m
(_18:Product {name: ' Carretera '}),"(Carretera)-[:SOLD_IN {sales: '$32,370.00 '}]->(Canada)",(_17:Country {name: 'Canada'})
(_18:Product {name: ' Carretera '}),"(Carretera)-[:SOLD_IN {sales: '$26,420.00 '}]->(Germany)",(_19:Country {name: 'Germany'})
(_18:Product {name: ' Carretera '}),"(Carretera)-[:SOLD_IN {sales: '$32,670.00 '}]->(France)",(_21:Country {name: 'France'})


In [11]:
graph.run("MATCH (n) WHERE n.name IS NOT NULL RETURN n")


n
(_16:Segment {name: 'Government'})
(_17:Country {name: 'Canada'})
(_18:Product {name: ' Carretera '})


In [12]:
graph.run("MATCH (n) RETURN n")

n
(_16:Segment {name: 'Government'})
(_17:Country {name: 'Canada'})
(_18:Product {name: ' Carretera '})


In [13]:
# !pip install networkx plotly
import networkx as nx
import plotly.graph_objects as go
# Create NetworkX graph
G = nx.Graph()


In [14]:
for node_data in graph.run("MATCH (n) WHERE n.name IS NOT NULL RETURN n"):
    node = node_data['n']
    G.add_node(node['name'], label=node['name'], title=str(node))  # Use 'name' as node ID and label

# Add edges
for rel_data in graph.run("MATCH (n)-[r]->(m) RETURN n, r, m"):
    source = rel_data["n"]["name"]
    target = rel_data["m"]["name"]
    rel_type = str(rel_data["r"].type)
    sales_value = rel_data["r"]["sales"]
    G.add_edge(source, target, label=f"{rel_type} (${sales_value})")

In [15]:
# Print graph information for debugging
print(f"Number of nodes: {G.number_of_nodes()}")
print(f"Number of edges: {G.number_of_edges()}")

Number of nodes: 16
Number of edges: 30


In [16]:
print("Sample Nodes:", list(G.nodes(data=True))[0:5])  # Print the first 3 nodes with their attributes

Sample Nodes: [('Government', {'label': 'Government', 'title': "(_16:Segment {name: 'Government'})"}), ('Canada', {'label': 'Canada', 'title': "(_17:Country {name: 'Canada'})"}), (' Carretera ', {'label': ' Carretera ', 'title': "(_18:Product {name: ' Carretera '})"}), ('Germany', {'label': 'Germany', 'title': "(_19:Country {name: 'Germany'})"}), ('Midmarket', {'label': 'Midmarket', 'title': "(_20:Segment {name: 'Midmarket'})"})]


In [17]:
# Print node information
for node in G.nodes():
    print(f"Node ID: {node}, Label: {G.nodes[node].get('label', 'No Label')}")


Node ID: Government, Label: Government
Node ID: Canada, Label: Canada
Node ID:  Carretera , Label:  Carretera 
Node ID: Germany, Label: Germany
Node ID: Midmarket, Label: Midmarket
Node ID: France, Label: France
Node ID: Mexico, Label: Mexico
Node ID:  Montana , Label:  Montana 
Node ID: Channel Partners, Label: Channel Partners
Node ID: Enterprise, Label: Enterprise
Node ID: Small Business, Label: Small Business
Node ID: United States of America, Label: United States of America
Node ID:  Paseo , Label:  Paseo 
Node ID:  Velo , Label:  Velo 
Node ID:  VTT , Label:  VTT 
Node ID:  Amarilla , Label:  Amarilla 


In [18]:
# Layout for nodes (adjust as needed)
# 4. Layout (Adjust if needed)
pos = nx.spring_layout(G, k=1, iterations=50)


In [19]:
# Create a Plotly figure
fig = go.Figure()

In [20]:
fig.add_trace(go.Scatter(
    x=[pos[node][0] for node in G.nodes()],
    y=[pos[node][1] for node in G.nodes()],
    mode='markers+text',
    text=list(G.nodes()),
    marker=dict(
        size=[15 * G.degree(node) for node in G.nodes()],
        color=['blue' if G.nodes[node]['label'].startswith('Product') else 'red' for node in G.nodes()]
    ),
    hoverinfo='text'
))

for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    fig.add_trace(go.Scatter(
        x=[x0, x1], y=[y0, y1],
        mode='lines+text',
        text=[edge[2]['label']],  # Get label from edge data
        textposition='middle center',
        line=dict(color='gray', width=1),
        hoverinfo='text'
    ))

fig.update_layout(
    title='Graph from Neo4j Data',
    titlefont_size=16,
    showlegend=False,
    hovermode='closest',
    margin=dict(b=20, l=5, r=5, t=40),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    width=1200, height=800,
)

fig.show()